In [1]:
!pip install pulp
from pulp import *

In [2]:
# Creates a list of the products
media = ['TV', 'Print', 'Web', 'SocialMedia']

# A dictionary of the cost per medium
cost = {'TV': 500, 
         'Print': 200,
         'Web': 250,
         'SocialMedia': 125}

# A dictionary of the reach per medium
reach = {'TV': 50, 
         'Print': 25,
         'Web': 20,
         'SocialMedia': 15}

# A dictionary of the max ads per medium
maxAds = {'TV': 20, 
         'Print': 15,
         'Web': 10,
         'SocialMedia': 15}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Media Planning", LpMinimize)

# A dictionary called 'production_vars' is created to contain the referenced Variables
x = LpVariable.dicts("x",media,0)

# Create objective function
prob += lpSum([cost[i]*x[i] for i in media]), "Total Cost of Media"

# Create Reach constraint
prob += lpSum([x[i]*reach[i] for i in media]) >= 1500, "Reach"

# Create max constraints
for m in media:
    prob += x[m] <= maxAds[m], "{}".format(m)

In [3]:
print(prob)

Media Planning:
MINIMIZE
200*m_Print + 125*m_SocialMedia + 500*m_TV + 250*m_Web + 0
SUBJECT TO
Reach: 25 m_Print + 15 m_SocialMedia + 50 m_TV + 20 m_Web >= 1500

TV: m_TV <= 20

Print: m_Print <= 15

Web: m_Web <= 10

SocialMedia: m_SocialMedia <= 15

VARIABLES
m_Print Continuous
m_SocialMedia Continuous
m_TV Continuous
m_Web Continuous



In [4]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

# Print the value of the variables at the optimum
for i in media:
    print("%s = %f" % (x[i].name, x[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

Status = Optimal
m_TV = 18.000000
m_Print = 15.000000
m_Web = 0.000000
m_SocialMedia = 15.000000
Objective = 13875.000000


In [8]:
# Create model with deviation constraints
import copy

# A dictionary of whether or not a medium is digital
channelType = {'TV': 1, 
         'Print': 1,
         'Web': -1,
         'SocialMedia': -1}

# Create Diversification constraints
prob += lpSum([x[i]*channelType[i] for i in media]) <= 5, "Diversification"
prob += lpSum([x[i]*(-1)*channelType[i] for i in media]) <= 5, "Diversification2"

In [9]:
prob

Media Planning:
MINIMIZE
200*m_Print + 125*m_SocialMedia + 500*m_TV + 250*m_Web + 0
SUBJECT TO
Reach: 25 m_Print + 15 m_SocialMedia + 50 m_TV + 20 m_Web >= 1500

TV: m_TV <= 20

Print: m_Print <= 15

Web: m_Web <= 10

SocialMedia: m_SocialMedia <= 15

Diversification: m_Print - m_SocialMedia + m_TV - m_Web <= 5

Diversification2: - m_Print + m_SocialMedia - m_TV + m_Web <= 5

VARIABLES
m_Print Continuous
m_SocialMedia Continuous
m_TV Continuous
m_Web Continuous

In [10]:
# Optimize

prob.solve()

# Print the value of the variables at the optimum
for i in media:
    print("%s = %f" % (x[i].name, x[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

m_TV = 14.285714
m_Print = 15.000000
m_Web = 9.285714
m_SocialMedia = 15.000000
Objective = 14339.285575
